In [10]:
import nltk
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import wordnet
import random

def handler(event, context):
    text = 'Hello! I\'ll repeat anything you say to me.'
    if 'request' in event and \
            'original_utterance' in event['request'] \
            and len(event['request']['original_utterance']) > 0:
        text = event['request']['original_utterance']
    return {
        'version': event['version'],
        'session': event['session'],
        'response': {
            'text': text,
            'end_session': 'false'
        },
    }


def lemmatization(list_of_words):
    lemmatizer = WordNetLemmatizer()
    pos = wordnet.VERB
    for word in list_of_words:
        word = lemmatizer.lemmatize(word, pos)
    return list_of_words

def filter_countries(text):
    tokens = nltk.word_tokenize(text)
    tokens = lemmatization(tokens)

    
def create_response(event, text):
    tokens = nltk.word_tokenize(text)
    if '?' in tokens:
        tokens = lemmatization(tokens)
        is_noun = lambda pos: pos[:2] == 'NN'
        nouns = [word for (word, pos) in nltk.pos_tag(tokenized) if is_noun(pos)]
        buttons = []
        for n in nouns:
            buttons.append({'title' : n})
    return {
        'version': event['version'],
        'session': event['session'],
        'response': {
            'text': text,
            'buttons' : buttons,
            'end_session': 'false'
        },
    }
 
    

In [11]:
#список стран
countries = {
    'Франция': ['Европа', 'культура', 'отдых', 'горы', 'лыжи', 'море', 'водоёмы', 'сёрфинг'],
    'Испания' : ['Европа', 'культура', 'отдых', 'горы', 'лыжи', 'море', 'ночная жизнь'],
    'Италия' : ['Европа', 'культура', 'отдых', 'горы', 'лыжи', 'море', 'водоёмы'],
    'Великобритания': ['Европа', 'культура', 'море', 'водоёмы',  'сёрфинг'],
    'Турция' : ['Европа', 'культура', 'отдых', 'море'],
    'Греция' : ['Европа', 'культура', 'отдых', 'море', 'ночная жизнь'],
    'Нидерланды' : ['Европа', 'культура', 'море'],
    'Швейцария' : ['Европа', 'лыжи', 'горы', 'водоёмы'],
    'Австрия' : ['Европа', 'культура', 'отдых', 'горы', 'лыжи','водоёмы'],
    'Германия' : ['Европа', 'культура', 'горы', 'лыжи', 'ночная жизнь', 'водоёмы'],
    
    
    'США' : ['Америка', 'море', 'горы', 'пустыня', 'лыжи', 'отдых', 
             'культура', 'дикая природа', 'водоёмы',  'сёрфинг'],
    'Мексика' : ['Америка', 'море', 'горы','пустыня','древность', 'ночная жизнь', 'дайвинг'],
    
    
    'Китай' : ['Азия', 'море', 'горы','культура', 'ночная жизнь','пустыня',
               'дикая природа', 'водоёмы',  'сёрфинг'],
    'Таиланд' : ['Азия', 'море', 'отдых', 'ночная жизнь','водоёмы'],
    'Сингапур' : ['Азия', 'культура', 'ночная жизнь'],
    
    'Марокко' : ['Африка', 'море', 'горы',  'пустыня', 'ночная жизнь'],
    'ЮАР' : ['Африка', 'море', 'горы', 'пустыня', 'дикая природа', 'дайвинг',  'сёрфинг'],
    'Тунис' : ['Африка', 'море', 'горы', 'пустыня', 'отдых'],
    'Египет' : ['Африка', 'море', 'пустыня', 'культура', 'отдых', 'дайвинг'],
    'Зимбабве' : ['Африка', 'горы', 'дикая природа'],
    'Алжир' : ['Африка', 'море', 'горы', 'пустыня'],
    'ОАЭ' : ['Африка', 'море', 'отдых', 'культура', 'дикая природа']
}


continents = ['Америка', 'Африка', 'Европа', 'Азия' ]


clasterizator = {
    
    'дикая природа' : ['дикая', 'природа', 'джунгли', "лес"],
    
    'водоёмы' : ['река', 'озеро'],
    
    'горы' : ['горы', 'водопад'],
    
    'пустыня' : ['пустыня'],
    
    'море' : ['море', 'океан'],
    
    
    
    'ночная жизнь' : ['бар', 'клуб', 'дискотека', 'тусовка', 'концерт', 'ночной'],
    
    'культура' : ['музей', 'театр', 'опера', 'балет', 'архитектура', 'церковь'],
    
    
    
    'отдых' : ['пляж', 'отель', 'бассейн', 'загорать', 'всё включено', 'отдых'],
    
    
    'лыжи' : ['лыжи', 'горнолыжный', 'сноуборд'],
    
    'дайвинг' : ['дайвинг'],
    
    'сёрфинг' : ['сёрфинг'] 
}





In [12]:
def reverse_map(m):
    d = {}
    for key, value in m.items():
        for item in value:
            d[item] = key
    return d

In [13]:
def county_mining(continents, keywords):
    candidats = {}
    for continent in continents:
        for country, features in countries.items():
            if continent in features:
                candidats[country] = features
    candidats_score = {country : 0 for country in candidats}
    max_score = 0
    for key in keywords:
        for country, features in candidats.items():
            if key in features:
                candidats_score[country] += 1
                max_score = max(candidats_score[country], max_score)
    best_countries = []
    for country, score in candidats_score.items():
        if score == max_score:
            best_countries.append(country)
    idx = 0
    if len(best_countries) > 1:
        idx = random.randint(0, len(best_countries)-1)
    return best_countries[idx]

In [18]:
result = county_mining(['Европа'], ['лыжи', 'водоёмы', 'ночная жизнь'])

In [19]:
print(result)

Германия
